In [17]:
import pandas as pd
url = "https://apisidra.ibge.gov.br/values/t/7060/n7/all/v/2265/p/all/c315/7169/d/v2265%202"
dados= pd.read_json(url + "?formato=json")
dados.head(3)


In [ ]:
dados.rename(columns=dados.iloc[0],inplace=True)
dados.columns

Index(['Nível Territorial (Código)', 'Nível Territorial',
       'Unidade de Medida (Código)', 'Unidade de Medida', 'Valor',
       'Região Metropolitana (Código)', 'Região Metropolitana',
       'Variável (Código)', 'Variável', 'Mês (Código)', 'Mês',
       'Geral, grupo, subgrupo, item e subitem (Código)',
       'Geral, grupo, subgrupo, item e subitem'],
      dtype='object')

In [16]:
dados.drop(index=[0],axis=0,inplace=True)


KeyError: '[0] not found in axis'

In [22]:
dados["data"] = pd.to_datetime(dados["Mês (Código)"], format="%Y%m")
dados = dados[dados["Valor"] != '...']
dados["Valor"] = dados["Valor"].astype(float)

In [23]:
dados['Região Metropolitana'].unique()

array(['Belém - PA', 'Fortaleza - CE', 'Recife - PE', 'Salvador - BA',
       'Belo Horizonte - MG', 'Grande Vitória - ES',
       'Rio de Janeiro - RJ', 'São Paulo - SP', 'Curitiba - PR',
       'Porto Alegre - RS'], dtype=object)

In [30]:
regioes = {
    "Norte": ["Belém - PA"],
    "Nordeste": ["Fortaleza - CE", "Recife - PE", "Salvador - BA"],
    "Sudeste": ["Belo Horizonte - MG", "Grande Vitória - ES", "Rio de Janeiro - RJ", "São Paulo - SP"],
    "Sul": ["Curitiba - PR", "Porto Alegre - RS"]
}
dados["Região"] = dados["Região Metropolitana"].apply(lambda metro: next((regiao for regiao, metros in regioes.items() if metro in metros), None))
dados["Metro"] = dados["Região Metropolitana"]


In [31]:
dados_tratados = dados[["data", "Região", "Metro", "Valor"]]
dados_tratados

,data,Região,Metro,Valor
12,2020-12-01,Norte,Belém - PA,4.63
13,2021-01-01,Norte,Belém - PA,4.18
14,2021-02-01,Norte,Belém - PA,5.42
15,2021-03-01,Norte,Belém - PA,6.44
16,2021-04-01,Norte,Belém - PA,7.32
...,...,...,...,...
516,2023-12-01,Sul,Porto Alegre - RS,4.63
517,2024-01-01,Sul,Porto Alegre - RS,4.53
518,2024-02-01,Sul,Porto Alegre - RS,4.29
519,2024-03-01,Sul,Porto Alegre - RS,2.86


In [70]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.line(
    dados_tratados,
    x="data",
    y="Valor",  
    color="Metro",
    facet_col="Região",  
    facet_col_wrap=2,  
    labels={
        "Valor": "Var. % anual",  
        "data": "Ano",
        "Metro": "Região Metropolitana",
    },
    title="Taxa de Inflação Regional no Brasil",
)

fig.update_xaxes(
    tickformat="%Y",
    dtick="M12",  # Define o intervalo dos ticks para 1 ano
)

# Obtem as regiões únicas e o número de colunas
regioes = dados_tratados['Região'].unique()
n_cols = 2  # definiu facet_col_wrap=2

# Cria as anotações para os títulos dos subplots
annotations = []
for i, regiao in enumerate(regioes):
    row = i // n_cols
    col = i % n_cols
    annotations.append(
        dict(
            text=regiao,
            x=col / n_cols + 0.5 / n_cols,  # Centraliza o título na coluna
            y=1 - row / (len(regioes) / n_cols),  # Posiciona o título acima do subplot
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=6, color="black"),
        )
    )

# Adiciona a anotação "Dados: IBGE" à lista annotations
annotations.append(
    dict(
        text="Dados: IBGE ",
        showarrow=False,
        xref="paper",
        yref="paper",
        x=0.5,
        y=-0.15,  # Posição da legenda
        xanchor="center",
        yanchor="bottom",
        font=dict(size=8),
    )
)

fig.update_layout(
    font=dict(size=15),
    width=1000,  # Largura em pixels
    height=500,  # Altura em pixels
    title_x=0.5,  # Centraliza o título
    showlegend=True,
    margin=dict(l=20, r=20, t=80, b=20),  # Ajusta as margens (aumenta o topo para os títulos)
    annotations=annotations,  # Adiciona as anotações 
)

fig.show()